In [1]:
import numpy as np
import math

def upsample(input, size=None, scale_factor=None, align_corners=None, recompute_scale_factor=None):
    def bilinear_interpolation(batch, channel, y, x):
        height = batch.shape[1]
        width = batch.shape[2]

        x1 = max(min(math.floor(x), width - 1), 0)
        y1 = max(min(math.floor(y), height - 1), 0)
        x2 = max(min(math.ceil(x), width - 1), 0)
        y2 = max(min(math.ceil(y), height - 1), 0)

        a = float(batch[channel, y1, x1])
        b = float(batch[channel, y2, x1])
        c = float(batch[channel, y1, x2])
        d = float(batch[channel, y2, x2])

        dx = x - x1
        dy = y - y1

        new_pixel = a * (1 - dx) * (1 - dy)
        new_pixel += b * dy * (1 - dx)
        new_pixel += c * dx * (1 - dy)
        new_pixel += d * dx * dy

        return new_pixel

    if isinstance(size, int):
        size = (size, size)
    elif isinstance(size, tuple) and len(size) == 1:
        size = (size[0], size[0])

    align_corners = True if align_corners is True else False
    recompute_scale_factor = True if recompute_scale_factor is True else False

    h_out, w_out = None, None
    result = []

    for batch in input:
        channels, h_in, w_in = batch.shape

        if recompute_scale_factor or h_out is None or w_out is None:
            if size is not None and scale_factor is not None:
                raise ValueError('one of size or scale_factor should be defined')
            elif size is None and scale_factor is None:
                raise ValueError('Size or scale factor should be specified!')
            elif size is not None and recompute_scale_factor:
                raise ValueError('recompute_scale_factor is not meaningful with an explicit size.')
            elif len(batch.shape) != 3:
                raise ValueError('Got {}D input, but bilinear mode needs 4D input'.format(len(batch.shape)))
            elif size is not None and len(size) == 2:
                h_out, w_out = size[0], size[1]
            else:
                h_out, w_out = math.floor(batch.shape[1] * scale_factor), math.floor(batch.shape[2] * scale_factor)

        out = np.zeros((channels, h_out, w_out), float)

        y_orig_center = (h_in - 1) / 2
        x_orig_center = (w_in - 1) / 2

        y_scaled_center = (h_out - 1) / 2
        x_scaled_center = (w_out - 1) / 2

        for c in range(channels):
            for y in range(h_out):
                for x in range(w_out):
                    if align_corners:
                        x_ = max(1, w_in - 1) / max(1, w_out - 1) * x
                        y_ = max(1, h_in - 1) / max(1, h_out - 1) * y
                    else:
                        x_ = (x - x_scaled_center) / scale_factor + x_orig_center
                        y_ = (y - y_scaled_center) / scale_factor + y_orig_center

                    out[c, y, x] = bilinear_interpolation(batch, c, y_, x_)

        result.append(out)

    return result

In [5]:
test_input = np.array([[[1, 2], [3, 4]]], dtype=float)

result = upsample([test_input], scale_factor=3, align_corners=True, recompute_scale_factor=False)

print(result)

[array([[[1. , 1.2, 1.4, 1.6, 1.8, 2. ],
        [1.4, 1.6, 1.8, 2. , 2.2, 2.4],
        [1.8, 2. , 2.2, 2.4, 2.6, 2.8],
        [2.2, 2.4, 2.6, 2.8, 3. , 3.2],
        [2.6, 2.8, 3. , 3.2, 3.4, 3.6],
        [3. , 3.2, 3.4, 3.6, 3.8, 4. ]]])]


In [6]:
import unittest
import numpy as np
from numpy.testing import assert_array_almost_equal

class UpsampleTests(unittest.TestCase):
    def test_upsample_scale_factor_2_align_corners(self):
        test_input = np.array([[[1, 2], [3, 4]]], dtype=float)
        result = upsample([test_input], scale_factor=2, align_corners=True, recompute_scale_factor=False)
        
        expected_output = np.array([[[1., 1.33333333, 1.66666667, 2.],
                                     [1.66666667, 2., 2.33333333, 2.66666667],
                                     [2.33333333, 2.66666667, 3., 3.33333333],
                                     [3., 3.33333333, 3.66666667, 4.]]])
        
        assert_array_almost_equal(result[0], expected_output)
        
    def test_upsample_scale_factor_3_align_corners(self):
        test_input = np.array([[[1, 2], [3, 4]]], dtype=float)
        result = upsample([test_input], scale_factor=3, align_corners=True, recompute_scale_factor=False)
        
        expected_output = np.array([[[1., 1.2, 1.4, 1.6, 1.8, 2.],
                                     [1.4, 1.6, 1.8, 2., 2.2, 2.4],
                                     [1.8, 2., 2.2, 2.4, 2.6, 2.8],
                                     [2.2, 2.4, 2.6, 2.8, 3., 3.2],
                                     [2.6, 2.8, 3., 3.2, 3.4, 3.6],
                                     [3., 3.2, 3.4, 3.6, 3.8, 4.]]])
        
        assert_array_almost_equal(result[0], expected_output)

In [7]:
unittest.main(argv=[''], exit=False)

..
----------------------------------------------------------------------
Ran 2 tests in 0.001s

OK
